In [0]:
from pyspark.ml.regression import LinearRegression, RandomForestRegressor
from pyspark.ml.evaluation import RegressionEvaluator
from pyspark.ml.feature import VectorAssembler

In [0]:
#importa
Carros_temp = spark.read.csv("/FileStore/tables/Carros.csv",inferSchema=True, header=True, sep=";")
Carros_temp.show(5)

+-------+---------+-----------+---------------+----+-----+---------+-----------+-------+-----------+---+
|Consumo|Cilindros|Cilindradas|RelEixoTraseiro|Peso|Tempo|TipoMotor|Transmissao|Marchas|Carburadors| HP|
+-------+---------+-----------+---------------+----+-----+---------+-----------+-------+-----------+---+
|     21|        6|        160|             39| 262| 1646|        0|          1|      4|          4|110|
|     21|        6|        160|             39|2875| 1702|        0|          1|      4|          4|110|
|    228|        4|        108|            385| 232| 1861|        1|          1|      4|          1| 93|
|    214|        6|        258|            308|3215| 1944|        1|          0|      3|          1|110|
|    187|        8|        360|            315| 344| 1702|        0|          0|      3|          2|175|
+-------+---------+-----------+---------------+----+-----+---------+-----------+-------+-----------+---+
only showing top 5 rows



In [0]:
#separa colunas
Carros = Carros_temp.select("Consumo","Cilindros","Cilindradas","HP")
Carros.show(5)

+-------+---------+-----------+---+
|Consumo|Cilindros|Cilindradas| HP|
+-------+---------+-----------+---+
|     21|        6|        160|110|
|     21|        6|        160|110|
|    228|        4|        108| 93|
|    214|        6|        258|110|
|    187|        8|        360|175|
+-------+---------+-----------+---+
only showing top 5 rows



In [0]:
#vetoriza atributos
veccaracteristicas = VectorAssembler(inputCols=[("Consumo"),("Cilindros"),("Cilindradas")],outputCol="caracteristicas")
Carros = veccaracteristicas.transform(Carros)
Carros.show(5)

+-------+---------+-----------+---+-----------------+
|Consumo|Cilindros|Cilindradas| HP|  caracteristicas|
+-------+---------+-----------+---+-----------------+
|     21|        6|        160|110| [21.0,6.0,160.0]|
|     21|        6|        160|110| [21.0,6.0,160.0]|
|    228|        4|        108| 93|[228.0,4.0,108.0]|
|    214|        6|        258|110|[214.0,6.0,258.0]|
|    187|        8|        360|175|[187.0,8.0,360.0]|
+-------+---------+-----------+---+-----------------+
only showing top 5 rows



In [0]:
#divide treino e teste
CarrosTreino, CarrosTeste = Carros.randomSplit([0.7,0.3])
print(CarrosTreino.count())
print(CarrosTeste.count())

24
8


In [0]:
#modelo
reglin = LinearRegression(featuresCol="caracteristicas", labelCol="HP")
modelo = reglin.fit(CarrosTreino)

In [0]:
#prever
previsao = modelo.transform(CarrosTeste)
previsao.show(5)

+-------+---------+-----------+---+-----------------+------------------+
|Consumo|Cilindros|Cilindradas| HP|  caracteristicas|        prediction|
+-------+---------+-----------+---+-----------------+------------------+
|     15|        8|        301|335| [15.0,8.0,301.0]|191.58783827553796|
|    104|        8|        472|205|[104.0,8.0,472.0]| 190.6522348700288|
|    133|        8|        350|245|[133.0,8.0,350.0]|190.88830412714617|
|    155|        8|        318|150|[155.0,8.0,318.0]| 190.8830881070078|
|    158|        8|        351|264|[158.0,8.0,351.0]|190.76865204550808|
+-------+---------+-----------+---+-----------------+------------------+
only showing top 5 rows



In [0]:
#avaliar performance
avaliar = RegressionEvaluator(predictionCol="prediction", labelCol="HP",metricName='rmse')
rmse = avaliar.evaluate(previsao)
print(rmse)

62.66042004151905
